In [ ]:
import datawrangling
import numpy as np
import scipy.stats as st
import pandas as pd
# i love python

In [ ]:
df = pd.read_excel("output.xlsx")
data,dict = datawrangling.dataWrangling(df,"Fall 2017","Spring 2018")
# data has old/new classes for each term

In [ ]:
def str_to_list(str):
    if type(str) != type("i love dynamic typing"):
        return [""]
    ret = str.replace("[", "").replace("\'", "").replace("]", "").split(", ")
    if ret[-1] == "":
        ret = ret[0:-1]
    return ret

df_newclasses = pd.DataFrame(columns=['old', 'new', 'added'])
for i in range(data.shape[0]):
    df_newclasses.loc[len(df_newclasses.index)] = [str_to_list(data.iloc[i][0]), str_to_list(data.iloc[i][1]), [x for x in str_to_list(data.iloc[i][1]) if x not in str_to_list(data.iloc[i][0])]]

pd.DataFrame.to_csv(df_newclasses, "ungabunga.csv")

In [ ]:
def get_class_cnt(ser):
    class_cnt = {}
    for key, class_list in ser.items():
        for c in class_list:
            if c not in class_cnt:
                class_cnt[c] = 1
            else:
                class_cnt[c] += 1
    return class_cnt

get_class_cnt(df_newclasses['added'])

In [ ]:
def get_bootstrap_sample(df):
    n = df.shape[0]
    boot_df = df.iloc[np.random.randint(n, size=n)]
    return boot_df

boot_results = []
trials = 100
for i in range(trials):
    boot_counts = get_class_cnt(get_bootstrap_sample(df_newclasses["added"]))
    
    boot_results.append(boot_counts)

def get_class_cnt_lists(boot_results):
    class_cnt_lists = {}
    for dic in boot_results:
        for key, value in dic.items():
            if key not in class_cnt_lists:
                class_cnt_lists[key] = [value]
            else:
                class_cnt_lists[key].append(value)

    return class_cnt_lists

In [41]:
def get_all_means(class_cnt_lists):
    mean_dict = {}
    for key, value in class_cnt_lists.items():
        mean_dict[key] = np.mean(value)
    return mean_dict

def get_all_stds(class_cnt_lists):
    mean_dict = {}
    for key, value in class_cnt_lists.items():
        mean_dict[key] = np.std(value)
    return mean_dict

means = get_all_means(get_class_cnt_lists(boot_results))
stds = get_all_stds(get_class_cnt_lists(boot_results))

print(means)
print(stds)

{'CSCI101': 28.71, 'CSCI358': 31.51, 'CSCI261': 65.61, 'CSCI306': 25.2, 'CSCI274': 22.09, 'CSCI400': 2.393258426966292, 'CSCI262': 46.46, 'CSCI341': 23.36, 'CSCI406': 5.84, 'CSCI442': 3.0927835051546393}
{'CSCI101': 4.705943051079135, 'CSCI358': 5.552467919763247, 'CSCI261': 7.1678378887918495, 'CSCI306': 4.442971978304613, 'CSCI274': 4.322256355192274, 'CSCI400': 1.5111419710970273, 'CSCI262': 5.598964189919418, 'CSCI341': 4.640086206095745, 'CSCI406': 2.6027677575995907, 'CSCI442': 1.4075091270825577}


In [42]:
def get_all_CIs(means, stds):
    CIs = {}
    for key, value in means.items():
        CIs[key] = (value - (1.96 * stds[key]), value + (1.96 * stds[key]))
    return CIs

get_all_CIs(means, stds)

{'CSCI101': (19.486351619884896, 37.9336483801151),
 'CSCI358': (20.627162877264038, 42.39283712273597),
 'CSCI261': (51.56103773796798, 79.65896226203202),
 'CSCI306': (16.491774922522957, 33.90822507747704),
 'CSCI274': (13.618377543823144, 30.561622456176856),
 'CSCI400': (-0.5685798363838814, 5.355096690316465),
 'CSCI262': (35.48603018775794, 57.43396981224206),
 'CSCI341': (14.26543103605234, 32.45456896394766),
 'CSCI406': (0.7385751951048025, 10.941424804895197),
 'CSCI442': (0.33406561607282637, 5.851501394236452)}